## <center> École Polytechnique de Montréal <br> Département Génie Informatique et Génie Logiciel <br>  INF8460 – Traitement automatique de la langue naturelle <br> </center>
## <center> TP1 INF8460 <br>  Automne 2021 </center>

## 1. DESCRIPTION
Dans ce TP, l’idée est d’effectuer de la recherche de passages de texte dans un corpus à partir d’une question en langue naturelle. Les questions et passages sont en anglais.

Voici un exemple : <br>
__Entrée : Question :__ What causes precipitation to fall?  

__Solution - Trouver un passage qui contient la réponse à la question :__ In meteorology, precipitation is any product of the condensation of atmospheric water vapor that falls under <mark> __gravity__ </mark>. The main forms of precipitation include drizzle, rain, sleet, snow, graupel and hail... Precipitation forms as smaller droplets coalesce via collision with other rain drops or ice crystals within a cloud. Short, intense periods of rain in scattered locations are called “showers”. 

Ici la réponse est en gras dans le texte.

## 2. LIBRARIES PERMISES
- Jupyter notebook
- NLTK
- Numpy 
- Pandas
- Sklearn
- Pour toute autre librairie, demandez à votre chargé de laboratoire


## 3. INFRASTRUCTURE

- Vous avez accès aux GPU du local L-4818. Dans ce cas, vous devez utiliser le dossier temp (voir le tutoriel VirtualEnv.pdf)

## 4. DESCRIPTION DES DONNEES

Dans ce projet, vous utiliserez le jeu de données dans le répertoire _data_. Il est décomposé en données d’entrainement (train), de validation (dev) et de test (test). <br>

Nous ne mettrons à votre disposition que les données d’entrainement et de validation. Les données de test ne contiennent pas le paragraphe de réponse et doivent être complétées avec les résultats de votre système.
Nous vous fournissons un ensemble de données qui comprend un corpus (_corpus.csv_) qui contient tous les passages et leurs identificateurs (ID) et un jeu de données qui associe une question, un passage, et une réponse qui est directement extraite du passage. Notez que certains passages contiennent des balises HTML et qu’il vous faudra procéder à un prétraitement de ces passages pour les enlever. <br>
Ce jeu de données est composé de trois sous-ensembles : 
- _Train_ : ensemble d’entraînement de la forme <QuestionID, QuestionText, PassageID, Réponse>. Le but est donc d’entrainer votre modèle à retrouver le passage qui contient la réponse à la question.
- _Validation_ : De la même forme que le Train, il vous permet de valider votre entraînement et de tester les performances de certains modules.  
- _Test_ : Un ensemble secret qui est utilisé pour évaluer votre système complet. Il est de la forme <QuestionID, Question>. Votre système doit trouver dans le corpus __corpus.csv__ le ou les passages les plus pertinents.

Notez qu’il est possible de répondre aux requis du TP sans utiliser la réponse à la question. C’est à vous de choisir si vous utilisez la réponse ou non. 

## 5. ETAPES DU TP 
A partir du notebook _inf8460_A21_TP1_ qui est distribué, vous devez réaliser les étapes suivantes. (Noter que les cellules dans le squelette sont là à titre informatif - il est fort probable que vous rajoutiez des sections au fur et à mesure de votre TP).

Ci-dessous définir la constante _PATH_ qui doit être utilisée par votre code pour accéder aux fichiers. Il est attendu que pour la correction, le chargé de lab n'ait qu'à changer la valeur de _PATH_ pour le répertoire où se trouver les fichiers de datasets.

In [3]:
""" # For ubuntu
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install numpy
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
"""
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soule\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soule\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\soule\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
PATH = "data/"

In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import numpy as np
import nltk

nltk.download("stopwords")
nltk.download("wordnet")
lem = WordNetLemmatizer()
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soule\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\soule\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 5.1. Pré-traitement (12 points)
Les passages et questions de votre ensemble de données doivent d’abord être représentés et indexés pour ensuite pouvoir effectuer une recherche de passage pour répondre à une question. On vous demande donc d’implémenter une étape de pré-traitement des données.
1) (_6 points_) Complétez les fonctions retournant les informations suivantes (une fonction par information, chaque fonction prenant en argument le corpus (passages, questions) composé d'une liste de phrases segmentées en jetons/tokens) :
    1. Le nombre total de jetons (mots non distincts)
    2. Le nombre total de mots distincts (les types qui constituent le vocabulaire)
    3. Les N mots les plus fréquents du vocabulaire (N est un paramètre avec une valeur par défaut de 10) ainsi que leur fréquence
    4. Le ratio jeton/type
    5. Le nombre total de lemmes distincts
    6. Le nombre total de racines (stems) distinctes

In [6]:
def total_non_distinct(corpus):
    total = 0
    for _i, paragraph in corpus.items():
        total += len(paragraph)
    return total
    
def total_distinct(corpus):
    words = set()
    for _i, paragraph in corpus.items():
        for word in paragraph:
            words.add(word)
    return len(words)

def most_frequent(corpus, n = 10):
    words = {}
    for _i, paragraph in corpus.items():
        for word in paragraph:
            if word in words:
                words[word] += 1
            else:
                words[word] = 1
    return list(sorted(words.items(), key=lambda temp: temp[1], reverse=True))[:n]
    
def ratio_jeton_type(corpus):
    return total_non_distinct(corpus)/total_distinct(corpus)

def total_lemme_distinct(corpus):
    lemmed = corpus.apply(lambda words: [lem.lemmatize(word) for word in words])
    return total_distinct(lemmed)
    
def total_stems_distinct(corpus):
    stem = corpus.apply(lambda words: [ps.stem(word) for word in words])
    return total_distinct(stem)

2. (_1 point_) Ecrivez une fonction explore_corpus() qui fait appel à toutes les fonctions en 1) et imprime leur résultat.


In [7]:
def explore_corpus(corpus):
    print("nombre total de jetons: ",total_non_distinct(corpus))
    print("Nombre total de mots distincts: ", total_distinct(corpus))
    print("Nombre total de mots distincts: ",most_frequent(corpus))
    print("Ratio jeton/type: ",ratio_jeton_type(corpus))
    print("Nombre total lemmes distincts: ",total_lemme_distinct(corpus))
    print("Nombre total de racines distincts: ",total_stems_distinct(corpus))
    

3. (_5 points_) Pour la suite du TP, vous devez effectuer le pré-traitement du corpus (questions, passages) en convertissant le texte en minuscules, en segmentant le texte, en supprimant les mots outils et en lemmatisant le texte. Chaque opération doit avoir sa fonction python si elle n’est pas déjà implantée dans la question 1) précédente.

In [8]:
corpus = pd.read_csv(PATH+"corpus.csv", usecols = [1])
questions = pd.read_csv(PATH+"train_ids.csv", usecols = [2])

In [9]:
corpus.head()

,paragraph
0,The Normans (Norman: Nourmands; French: Norman...
1,"The Norman dynasty had a major political, cult..."
2,"The English name ""Normans"" comes from the Fren..."
3,"In the course of the 10th century, the initial..."
4,"Before Rollo's arrival, its populations did no..."


In [10]:
questions.head()

,question
0,Who leaders the sub-divisions of offices or di...
1,Besides using 3kV DC what other power type is ...
2,How many other cities had populations larger t...
3,Did von Neumann rule hidden variable theories?
4,What is the name of the book that has the laws...


In [11]:
paragraphs = corpus["paragraph"].apply(word_tokenize)
paragraphs.head()

0    [The, Normans, (, Norman, :, Nourmands, ;, Fre...
1    [The, Norman, dynasty, had, a, major, politica...
2    [The, English, name, ``, Normans, '', comes, f...
3    [In, the, course, of, the, 10th, century, ,, t...
4    [Before, Rollo, 's, arrival, ,, its, populatio...
Name: paragraph, dtype: object

In [12]:
explore_corpus(paragraphs)

nombre total de jetons:  11922724
Nombre total de mots distincts:  213164
Nombre total de mots distincts:  [('the', 595302), (',', 571498), ('>', 410243), ('<', 410182), ('.', 383281), ('of', 307955), ('and', 270716), ('in', 219592), ('to', 174688), ('a', 166063)]
Ratio jeton/type:  55.932164905894055
Nombre total lemmes distincts:  203790
Nombre total de racines distincts:  155592


In [13]:
def transform_min(corpus):
    return corpus.str.lower()

def tokenize(corpus):
    return corpus.apply(word_tokenize)

def removeStopWords(corpus):
    stop_words = set(stopwords.words("english"))
    return corpus.apply(lambda words: [word for word in words if word not in stop_words])

def lemmatize(corpus):
    lem = WordNetLemmatizer()
    return corpus.apply(lambda words: [lem.lemmatize(word) for word in words])

def clean_data(corpus):
    corpus = transform_min(corpus)
    corpus = tokenize(corpus)
    corpus = removeStopWords(corpus)
    corpus = lemmatize(corpus)
    return corpus

In [14]:
corpus = clean_data(corpus["paragraph"])

In [15]:
questions = clean_data(questions["question"])

In [16]:
corpus.head()

0    [norman, (, norman, :, nourmands, ;, french, :...
1    [norman, dynasty, major, political, ,, cultura...
2    [english, name, ``, norman, '', come, french, ...
3    [course, 10th, century, ,, initially, destruct...
4    [rollo, 's, arrival, ,, population, differ, pi...
Name: paragraph, dtype: object

### 5.2. Représentation de questions et de passages (14 points)

1. (_10 points_) En utilisant sklearn et à partir de votre corpus pré-traité, vous devez implanter un modèle M1 qui est de représenter chaque passage et question avec votre vocabulaire, en utilisant un modèle sac de mots des n-grammes (n=1) qu’ils contiennent et en pondérant ces éléments avec TF-IDF. Notez que les questions doivent aussi être inclues dans la construction du vocabulaire.

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances


In [18]:
paragraphs_t = corpus.apply(lambda x: " ".join(x))
questions_t = questions.apply(lambda x: " ".join(x))

In [19]:
questions_paragraph = pd.concat([paragraphs_t, questions_t], axis=0)

In [20]:
def tfidf_model(model=1):
  vectorizer1 = TfidfVectorizer(ngram_range=(1, model))
  tfidf = vectorizer1.fit(questions_paragraph.values)
  return vectorizer1.transform(questions_t), vectorizer1.transform(paragraphs_t)


In [21]:
tfidf_questions1, tfidf_paragraphs1 = tfidf_model()

2. (_4 points_) Expérimentez maintenant avec un modèle n-gramme (n=1,2) mélangeant les unigrammes et les bigrammes et pondéré avec TF-IDF.

In [22]:
tfidf_questions2, tfidf_paragraphs2 = tfidf_model(model=2)

Pour M1 et M2, assurez-vous de réutiliser la même fonction avec comme paramètre les n-grammes à considérer.

### 5.3. Ordonnancement des passages (10 points)
Maintenant que vous avez une représentation de vos passages et questions, il faut être capable de déterminer quel passage sera le plus pertinent pour la question posée. Il vous faut donc retrouver un top-N (N=1,5,10 … ) de passages utiles pour répondre à la question. Ces passages devront être ordonnés du plus pertinent au moins pertinent. Idéalement le passage à la position 1 sera celui qui contient la réponse à la question.
<br>
<br>
Vous devez écrire des fonctions pour évaluer la similarité entre la représentation de la question et celle de chaque passage et retourner les N passage les plus similaires où N est un paramètre. 
1. (_5 points_) En utilisant la distance euclidienne
2. (_5 points_) En utilisant la distance cosinus


In [23]:
def euclidean_distances_per_question(question, n=1):
  tfidfs = euclidean_distances(question, tfidf_paragraphs1)[0]
  return np.argsort(tfidfs)[:n]


In [24]:
def cosinus_distances_per_question(question, n=1):
  tfidfs = cosine_distances(question, tfidf_paragraphs1)[0]
  return np.argsort(tfidfs)[:n]


In [25]:
euclidean_distances_per_question(tfidf_questions1[9,], 30)

array([ 6642, 83055, 81077, 27041, 17797,  5771, 74938, 23824, 33772,
       81269,  7544, 81733,  6648, 65286, 27925, 50161, 50907, 39294,
       54172, 53859, 18876, 53154, 54844, 18538, 28877, 16814, 44444,
       59456, 33732, 80360], dtype=int64)

In [26]:
cosinus_distances_per_question(tfidf_questions1[9,], 30)

array([ 6642, 83055, 81077, 27041, 17797,  5771, 74938, 23824, 33772,
       81269,  7544, 81733,  6648, 65286, 27925, 50161, 50907, 39294,
       54172, 53859, 18876, 53154, 54844, 18538, 28877, 16814, 44444,
       59456, 33732, 80360], dtype=int64)

### 5.4. Évaluation (15 points)
En utilisant votre ensemble de validation : <br>
1. (_5 points_) Vous devez calculer la précision top-N (N=1,5,10, 50) de votre modèle M1 et M2 avec la distance euclidienne et cosinus et les afficher. 


In [27]:
eval = pd.read_csv(PATH+"val_ids.csv", usecols = [1, 3])

In [28]:
def precision(data, model=1, n=1):
  nbr = 0
  for index, row in data.iterrows():
    if row["paragraph_id"] in cosinus_distances_per_question(tfidf_questions1[row["id"]], n):
      nbr += 1
  return nbr / data.shape[0], data.shape[0], nbr

In [29]:
a, b, c = precision(eval)
print(a)
print(b)
print(c)

0.00012232415902140674
8175
1


2. (_5 points_) Pour chacun de ces modèles, générez une courbe de performance faisant varier le N (N=1, 5, 10, 50)

3. (_5 points_) A cette étape, vous devez produire un fichier _passage_submission_M1.csv_ et _passage_submission_M2.csv_ qui contient pour toutes les questions de l’ensemble de test le top-N des passages retournés par votre modèle M1 et M2 pour y répondre. C’est à vous de déterminer si vous utiliserez la distance euclidienne ou cosinus basé sur vos résultats d’évaluation sur l’ensemble de validation en 1) et 2). Le fichier doit respecter le format suivant pour chaque top_N(N=1,5,10,50) :  <QuestionID, PassageID1 ;… ;PassageIDN>. Le format est démontré dans _sample_passage_submission.csv_.

### 5.5. Le plus (24 points)

1. (_21 points_) Vous devez proposer un modèle M3 différent (basé sur l’apprentissage machine par exemple) afin de déterminer un score de pertinence d’un passage pour une question donnée et ordonner les passages. 
    - Faites une petite recherche sur l’état de l’art en consultant https://nlp.stanford.edu/IR-book/information-retrieval-book.html
    - Vous êtes libres de proposer une autre métrique de poids, ou une autre façon d’ordonner les passages (exemple : méthodes de type _learning to rank_) et de partir de votre corpus initial ou de votre ordonnancement en M1/M2 (choisissez le meilleur) et de réordonnancer les passages obtenus par votre premier modèle.
    - Expliquez votre modèle et son intérêt dans votre notebook. Le nombre de points obtenus dépendra de l’effort mis dans cette partie.

## Explication de ce que allons faire

Nous allons utiliser plusieurs modèles de langage naturel connus pour répondre à l'objectif de pouvoir associé le bon extrait de paragraphe à la bonne question. De plus, nous allons expliquer comment est-ce que les différents modèles fonctionnent. Ensuite, nous allons comparer leurs résultats entre eux et avec le TF-IDF et expliquer pourquoi certains modèles ont performés mieux que d'autres.

Les différents modèles que nous allons explorer sont les suivants :
- word2vec
- fastText


In [20]:
import gensim

train_paragraphs = pd.read_csv(PATH + "corpus.csv", usecols = [1])["paragraph"]
train_questions = pd.read_csv(PATH + "train_ids.csv", usecols = [2])["question"]
    
train_processed_paragraphs = train_paragraphs.apply(lambda x : gensim.utils.simple_preprocess(x))
train_processed_questions = train_questions.apply(lambda x : gensim.utils.simple_preprocess(x))

0    [the, normans, norman, nourmands, french, norm...
1    [the, norman, dynasty, had, major, political, ...
2    [the, english, name, normans, comes, from, the...
3    [in, the, course, of, the, th, century, the, i...
4    [before, rollo, arrival, its, populations, did...
Name: paragraph, dtype: object
<class 'pandas.core.series.Series'>

0    [who, leaders, the, sub, divisions, of, office...
1    [besides, using, kv, dc, what, other, power, t...
2    [how, many, other, cities, had, populations, l...
3    [did, von, neumann, rule, hidden, variable, th...
4    [what, is, the, name, of, the, book, that, has...
Name: question, dtype: object


## word2vec


In [21]:
from gensim.models import Word2Vec


word2vec_model = Word2Vec(train_processed_paragraphs.to_list(), vector_size = 200, window=5)

def vectorize_word2vec(sentence):
    ans = []
    for word in sentence:
        if word2vec_model.wv.has_index_for(word):
            ans.append(word2vec_model.wv[word])
    return np.array(ans)

Done !


In [54]:
train_vec_paragraphs = train_processed_paragraphs.apply(vectorize_word2vec)
train_vec_paragraphs_sentence = train_vec_paragraphs.apply(lambda x : np.mean(x ,axis = 0))

train_vec_questions = train_processed_questions.apply(vectorize_word2vec)
train_vec_questions_sentence = train_vec_questions.apply(lambda x : np.mean(x ,axis = 0))






### FastText
   

In [57]:
from gensim.models import FastText

fasttext_model = FastText(train_processed_paragraphs.to_list(), vector_size = 200, window=5)

def vectorize_fasttext(sentence):
    ans = []
    for word in sentence:
        if fasttext_model.wv.has_index_for(word):
            ans.append(fasttext_model.wv[word])
    return np.array(ans)

In [ ]:
train_vec_paragraphs_fasttext = train_processed_paragraphs.apply(vectorize_fasttext)
train_vec_paragraphs_sentence_fasttext = train_vec_paragraphs_fasttext.apply(lambda x : np.mean(x ,axis = 0))

train_vec_questions_fasttext = train_processed_questions.apply(vectorize_fasttext)
train_vec_questions_sentence_fasttext = train_vec_questions_fasttext.apply(lambda x : np.mean(x ,axis = 0))



2. (_2  point_) Vous devez ensuite afficher l’évaluation de votre modèle M3 tel que décrit dans la section 5.4 Evaluation en utilisant les mêmes fonctions. Notamment, vous devez comparer les performances de vos modèles M1, M2 et M3 sur l’ensemble de validation avec une courbe de performance faisant varier le N (N=1, 5, 10, …)

3. (_1 point_) En utilisant votre modèle M3, vous devez produire un fichier passage_submission_M3.csv qui contient pour toutes les questions de l’ensemble de test le top-N des passages retournés par votre système pour y répondre. Le fichier doit respecter le format suivant pour chaque top_N (N=1,5,10,50) :  <QuestionID, PassageID1…PassageIDN>. _Le format est démontré dans sample_passage_submission.csv_

## LIVRABLES
Vous devez remettre sur Moodle:
1. _Le code_ : Un Jupyter notebook en Python qui contient le code implanté avec les librairies permises. Le code doit être exécutable sans erreur et accompagné des commentaires appropriés dans le notebook de manière à expliquer les différentes fonctions et étapes dans votre projet. Nous nous réservons le droit de demander une démonstration ou la preuve que vous avez effectué vous-mêmes les expériences décrites. _Attention, en aucun cas votre code ne doit avoir été copié d’une quelconque source_. Les critères de qualité tels que la lisibilité du code et des commentaires sont importants. Tout votre code et vos résultats doivent être exécutables et reproductibles ; 
2. Un fichier _requirements.txt_ doit indiquer toutes les librairies / données nécessaires ;
3. Un lien _GoogleDrive_ ou similaire vers les modèles nécessaires pour exécuter votre notebook si approprié ;
4. Les fichiers de soumission de données de test _passage_submission_M1.csv_ et _passage_submission_M2.csv_
5. Un document _contributions.txt_ : Décrivez brièvement la contribution de chaque membre de l’équipe. Tous les membres sont censés contribuer au développement. Bien que chaque membre puisse effectuer différentes tâches, vous devez vous efforcer d’obtenir une répartition égale du travail. En particulier, tous les membres du projet devraient participer à la conception du TP et participer activement à la réflexion et à l’implémentation du code.

## EVALUATION 
Votre TP sera évalué selon les critères suivants :
1. Exécution correcte du code
2. Performance correcte des modèles
3. Organisation du notebook
4. Qualité du code (noms significatifs, structure, performance, gestion d’exception, etc.)
5. Commentaires clairs et informatifs

## CODE D’HONNEUR
- Règle 1:  Le plagiat de code est bien évidemment interdit.
- Règle 2: Vous êtes libres de discuter des idées et des détails de mise en œuvre avec d'autres équipes. Cependant, vous ne pouvez en aucun cas consulter le code d'une autre équipe INF8460, ou incorporer leur code dans votre TP.
- Règle 3:  Vous ne pouvez pas partager votre code publiquement (par exemple, dans un dépôt GitHub public) tant que le cours n'est pas fini.
